In [35]:
import os
import sys

In [36]:
%pwd

'c:\\Users\\priya\\OneDrive\\Desktop\\Kidney Disease Classification'

In [37]:
os.chdir("../")

In [38]:
%pwd

'c:\\Users\\priya\\OneDrive\\Desktop\\Kidney Disease Classification'

In [39]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [40]:
from src.kidneyDiseaseClassifier.constants import *
from src.kidneyDiseaseClassifier.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        # print(config)
        
        # print(config.root_dir)

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [42]:
import os
import zipfile
import gdown
from src.kidneyDiseaseClassifier import logger
from src.kidneyDiseaseClassifier.utils.common import get_size

In [43]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [44]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    print(data_ingestion_config)
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-22 11:21:58,916: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-22 11:21:58,920: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-22 11:21:58,922: INFO: common: created directory at: artifacts]
[2023-10-22 11:21:58,924: INFO: common: created directory at: artifacts/data_ingestion]
{'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}
artifacts/data_ingestion
DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')
[2023-10-22 11:21:58,926: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into fil

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=18ff06d8-e5be-47c3-b6d7-25a81a37cac3
To: c:\Users\priya\OneDrive\Desktop\Kidney Disease Classification\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:15<00:00, 3.79MB/s]

[2023-10-22 11:22:17,422: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
